## SOFE3720 | FinalProject - Neighbourhoods in Toronto

## Table of Contents
* [Introduction](#introduction)
    * [Background](#background)
    * [Business Problem](#businessproblem)
* [Methodology](#methodology)
* [Data Sources](#data)



## **Introduction** <a name="introduction"></a>

**1.1. Background** <a name="background"></a>

**1.2. Business Problem** <a name="businessproblem"></a>


## **Methodology** <a name="methodology"></a>
This report aims to analyze the neighborhoods of Toronto city from different data sets and find the 
perfect spot. The following datasets will be utilized in the project: 

1) **Neighbourhoods Dataset:** This dataset contains neighborhood names as well as the geographic 
coordinates (latitude and longitude). The geographic coordinates will be used for two purposes; visualize 
Toronto map and call Foursquare API. 
 
2) **FourSquare:** Foursquare API is used to collect data to find the most common venues within a specific 
radius of a given geographic coordinate. 
 
3) **Neighborhood Profile Toronto:** This dataset contains the data for each of City of Toronto 
neighbourhoods. 
 
4) **Neighbourhood Crime Rates:** This dataset contains Crime Data by Neighbourhood. Data includes four- 
year averages and crime rates per 100,000 people by neighbourhood based on 2016 Census Population. 

5) **Wikipedia page to get more information about Toronto:** information we need to explore and cluster 
the  neighborhoods  in  Toronto.  You  will  scrape  the  Wikipedia  page,  wrangle,  and  clean  the  data,  and 
then read it into a pandas data frame. 

## **Data Sources** <a name="data"></a>


### Importing Libraries

The following csv files must be placed locally. This can be done by using a simple `wget` command to be able to access the required dataset.

In [226]:
%%capture
!wget -O GeoSpatial_Data https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv
!wget -O Neighbourhood_Profile https://ckan0.cf.opendata.inter.prod-toronto.ca/en_AU/datastore/dump/ef0239b1-832b-4d0b-a1f3-4153e53b189e?bom=True
!wget -O Crime_Data https://opendata.arcgis.com/datasets/af500b5abb7240399853b35a2362d0c0_0.csv

The following dependencies and libraries will be required before going forward and making sure the following codes work properly.

In [227]:
import pandas as pd     # library for data analysis          
import numpy as np      # library to handle data in a vectorized manner
import plotly.express as px

import folium           # library for map rendering
import requests         # library to handle request

from bs4 import BeautifulSoup as bs     
from geopy.geocoders import Nominatim   # Module to convert an address into latitude and longitude values

print("Libraries imported...")

Libraries imported...


## **1. Extracting Postal Code, Borough, Neighbourhood, Longitude, and, Latitude** <a name="extracting"></a>
### Scraping from Wikipedia page for Data
The table from the Wikipage has a list of all the Neighbourhoods in Toronto with the following Postal Code and associated Borough. It is scaped using and inserted into a dataframe using the code below:

In [228]:
# Requestion data from html url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_table_data = requests.get(url).text 
soup = bs(html_table_data, 'html5lib')

# Create dataframe with following columns (Postal, Borough, Neighbourhood)
df = pd.DataFrame(columns = ['PostalCode','Borough','Neighbourhood'])
# Scrape the Wikipedia page for the rows in the table
tb_rows = soup.find('table').tbody.find_all('tr')       

# Filtering the scraped data and inserting to dataframe
for rows in tb_rows :
    for column in rows.find_all('td') :
        if column.span.text != 'Not assigned' :
            span  = column.span.text.split('(')
            df = df.append({'PostalCode' : column.b.text,
                              'Borough' : span[0],
                              'Neighbourhood' : span[1][:-1]}, ignore_index=True)

# Replace the following name of borough
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

# Sort dataframe by PostalCode and reset to default indexing
df = df.sort_values('PostalCode').reset_index(drop = True)
df.head()   # print the first 5 in df

# df.shape    # shape/size of dataframe


C:\Users\rnlclng.pilar\AppData\Local\Temp\ipykernel_41424\2690276843.py:16: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\rnlclng.pilar\AppData\Local\Temp\ipykernel_41424\2690276843.py:16: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\rnlclng.pilar\AppData\Local\Temp\ipykernel_41424\2690276843.py:16: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\rnlclng.pilar\AppData\Local\Temp\ipykernel_41424\2690276843.py:16: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\rnlclng.pilar\AppData\Local\Temp\ipykernel_41424\2690276843.py:16: FutureWarning:

The frame.append method is deprecated and will be removed from p

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Extracting Latitude and Longitude 
One of the download file (`GeoSpatial_Data`) from earlier will be used and inserted to a seperate dataframe.

In [229]:
%%capture   
# wget the following dataset and store locally
!wget -O GeoSpatial_Data https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv

In [230]:
geospatial_data = pd.read_csv('GeoSpatial_Data')                    # Read from the csv file
geospatial_data.columns = ['PostalCode', 'Latitude', 'Longitude']   # Set the columns
geospatial_data.head()   # print the first 5 in df

# geospatial_data.shape    # shape/size of dataframe


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Joining Dataframes based on Postal Code
We must clean the data first and seperate the neighbourhood into separate rows from the first dataframe, before continuing to join the two dataframes

In [231]:
# Cleaning data to split and splitting neighbourhoods
df = df.assign(Neighbourhood=df.Neighbourhood.str.split(" / ")).explode('Neighbourhood')

# Join both data based on PostalCode
df = df.join(geospatial_data.set_index('PostalCode'), on = 'PostalCode')        

df.head() # print the first 5 in df


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern,43.806686,-79.194353
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
1,M1C,Scarborough,Port Union,43.784535,-79.160497
1,M1C,Scarborough,Highland Creek,43.784535,-79.160497


## **2. Exploring Toronto Neighbourhoods on a Map** <a name="map"></a>
### Creating Clustered Map of Toronto Neighbourhoods

In [232]:
df.Borough.value_counts()      # return most frequent-occuring Borough (most neighbourhood)

Etobicoke                 44
Scarborough               38
North York                36
Downtown Toronto          35
Central Toronto           16
West Toronto              13
Etobicoke Northwest        9
York                       8
East Toronto               6
East York                  5
East York/East Toronto     1
Downtown Toronto Stn A     1
Queen's Park               1
Mississauga                1
East Toronto Business      1
Name: Borough, dtype: int64

In [233]:
# Use geopy library to get the latitude and longitude values of Toronto city
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent = 'ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('Geographical Coordinates of Toronto City:')
print('Latitude: ', latitude)
print('Longitude: ', longitude)

Geographical Coordinates of Toronto City:
Latitude:  43.6534817
Longitude:  -79.3839347


Using Folium library we are able to create a map of Toronto using the aquired latitude and longitude values. Neighbourhoods can be mapped out from their coordinate values, as seen on the map below with the blue coloured dots.

In [234]:
# Array of Toronto boroughs
borough_array = ['North York', 'York ', 'East York', 'Downtown Toronto', 'Central Toronto', 'West Toronto', 'East Toronto', 'Downtown Toronto Stn A' , 'East Toronto Business', 'East York/East Toronto', 'Scarborough',
                 'Etobicoke', 'Etobicoke Northwest', "Queen's Park", 'Mississauga']

# Make changes in the dataframe accordingly
df1 = df.copy()
for boroughs in borough_array :
    for borough in boroughs :
        df1.replace(borough, str(boroughs), inplace = True)

colors_array = np.empty(15, dtype = str)
colors_array.fill('blue')

# cCeate map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add markers to map
for borough, color in zip(borough_array, colors_array) :
    df2 = df1[df1.Borough == str(borough)]
    for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius = 5,
            popup = label,
            color = 'blue',
            fill = True,
            fill_color = 'blue',
            fill_opacity = 1,
            parse_html = False).add_to(map_toronto)  
    
map_toronto


## **3. Correlation Between *'Crime Rate Vs. Unemployment, House Income, and Age Population'* in Toronto Neighbourhoods** <a name="correlation"></a>

### Neighbourhood Profiles
The following dataset shows detailed information about each neighbourhood in Toronto. This includes age, income, education, unemployment rate, and more. We will read this dataset from the one of the downloaded files from earlier.

In [235]:
%%capture   
# wget dataset and store locally
!wget -O Neighbourhood_Profile https://ckan0.cf.opendata.inter.prod-toronto.ca/en_AU/datastore/dump/ef0239b1-832b-4d0b-a1f3-4153e53b189e?bom=True

In [236]:
# Obtain the row number for "Unemployment" to allow us extract it from the dataframe
neighbourhood_profile = pd.read_csv('Neighbourhood_Profile')
neighbourhood_profile.head()

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,...,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,...,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,4,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177",...,"15,004","21,343","53,350","11,703","7,826","13,986","10,578","11,652","27,713","14,687"
4,5,Population,Population and dwellings,Census Profile 98-316-X2016001,Population Change 2011-2016,4.50%,-3.90%,8.00%,1.30%,4.60%,...,12.90%,3.80%,0.30%,7.20%,0.50%,2.60%,11.70%,7.50%,-0.40%,0.80%


The dataset `Neighbourhood_Profile` is large dataset of Toronto Neighbourhoods with the shape/size of:


In [237]:
neighbourhood_profile.shape     # return shape of dataset

(2383, 146)

### **PART 1 - Crime Rate (2019)**

From the dataset downloaded in the beginning (`Crime_Data`) we are able to find the types of Crime Rates in each neighbourhoods

In [238]:
%%capture
# wget the dataset and store it locally
!wget -O Crime_Data https://opendata.arcgis.com/datasets/af500b5abb7240399853b35a2362d0c0_0.csv

In [239]:
# Read from the csv data file
crime_data = pd.read_csv('Crime_Data')          

# Filter crime data to extract the following column
crime_data = crime_data[["Neighbourhood", "Hood_ID", "Population", 
                        "Assault_Rate_2019", "AutoTheft_Rate_2019", 
                        "BreakandEnter_Rate_2019", "Homicide_Rate_2019", 
                        "Robbery_Rate_2019", "TheftOver_Rate_2019"]]

# Rename Columns 
crime_data.rename(columns={'Hood_ID':'Neighbourhood ID'},inplace=True)
crime_data.rename(columns={'Assault_Rate_2019':'Assault Rate'},inplace=True)
crime_data.rename(columns={'AutoTheft_Rate_2019':'AuthoTheft Rate'},inplace=True)
crime_data.rename(columns={'BreakandEnter_Rate_2019':'BreakAndEnter Rate'},inplace=True)
crime_data.rename(columns={'Homicide_Rate_2019':'Homicide Rate'},inplace=True)
crime_data.rename(columns={'Robbery_Rate_2019':'Robbery Rate'},inplace=True)
crime_data.rename(columns={'TheftOver_Rate_2019':'TheftOver Rate'},inplace=True)

# Merge data based on Neighbourhood
# df = df.merge(crime_data.set_index('Neighbourhood'), on = 'Neighbourhood')

crime_data.head() # print the first 5 rows in df

,Neighbourhood,Neighbourhood ID,Population,Assault Rate,AuthoTheft Rate,BreakAndEnter Rate,Homicide Rate,Robbery Rate,TheftOver Rate
0,Yonge-St.Clair,97,12528,295.3,47.9,223.5,0.0,31.9,47.9
1,York University Heights,27,27593,1340.9,521.9,391.4,0.0,286.3,101.5
2,Lansing-Westgate,38,16164,445.4,198.0,241.3,0.0,68.1,68.1
3,Yorkdale-Glen Park,31,14804,1411.8,412.1,567.4,6.8,283.7,195.9
4,Stonegate-Queensway,16,25051,327.3,135.7,255.5,0.0,87.8,16.0


To get the total crime rate we must sum up all the different types of crime rate

In [240]:
# declare which column to deal with
columns = ["Assault Rate", "AuthoTheft Rate", "BreakAndEnter Rate", "Homicide Rate", "Robbery Rate", "TheftOver Rate"]

crime_data['Total Crime Rate'] = crime_data[columns].sum(axis=1)    # sum up all rates to find total
crime_data.drop(columns, axis=1, inplace=True)                      # drop columns 

crime_data.head() # print the first 5 rows in df

,Neighbourhood,Neighbourhood ID,Population,Total Crime Rate
0,Yonge-St.Clair,97,12528,646.5
1,York University Heights,27,27593,2642.0
2,Lansing-Westgate,38,16164,1020.9
3,Yorkdale-Glen Park,31,14804,2877.7
4,Stonegate-Queensway,16,25051,822.3


### **PART  2 - Unemployment Rate (2016)**

We can extract the Unemployment Rate from the following dataset (`Neighbourhood Profile`)

In [241]:
# Slice demographics dataframe to obtain "Unemployment" per Neighbourhood
neighbourhood_profile.index[neighbourhood_profile['Characteristic'] == 'Unemployment rate'].tolist()
slice_neighbourhood_profile = neighbourhood_profile.iloc[lambda df: [0, 1890], 4:]
slice_neighbourhood_profile.head()

,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,...,37,7,137,64,60,94,100,97,27,31
1890,Unemployment rate,8.2,9.8,9.8,6.1,6.7,7.2,7.2,10.2,7.7,...,9.8,8.5,10.6,7.7,6.6,5.2,6.9,5.9,10.7,8


Cleaning up data...

In [242]:
# Drop irrelevant columns
slice_neighbourhood_profile.drop(labels='City of Toronto',axis=1, inplace=True)
slice_neighbourhood_profile.rename(columns={'Characteristic':'Neighbourhood'}, inplace=True)
# Set index and Transpose
slice_neighbourhood_profile=slice_neighbourhood_profile.set_index('Neighbourhood').T
slice_neighbourhood_profile.reset_index(inplace = True)
# Re-order columns
slice_neighbourhood_profile.columns = ['Neighbourhood', 'Neighbourhood ID', 'Unemployment Rate']
# Set Neighbourhood ID and Unemployment Rate to numeric type
slice_neighbourhood_profile['Neighbourhood ID']=slice_neighbourhood_profile['Neighbourhood ID'].apply(pd.to_numeric) 
slice_neighbourhood_profile['Unemployment Rate']=slice_neighbourhood_profile['Unemployment Rate'].apply(pd.to_numeric) 
slice_neighbourhood_profile.head()

,Neighbourhood,Neighbourhood ID,Unemployment Rate
0,Agincourt North,129,9.8
1,Agincourt South-Malvern West,128,9.8
2,Alderwood,20,6.1
3,Annex,95,6.7
4,Banbury-Don Mills,42,7.2


### **PART 3 - Household Income (2015)**

We can extract the Household Income from the following dataset (`Neighbourhood Profile`)

In [243]:
# Obtain the row number for "Household Income" to allow us extract it from the dataframe
neighbourhood_profile.index[neighbourhood_profile['Characteristic'] == ('  Average after-tax income of households in 2015 ($)', 
                                                                        'Total - Household after-tax income groups in 2015 for private households - 100% data',
                                                                        '  Under $5,000',
                                                                        '  $5,000 to $9,999',
                                                                        '  $10,000 to $14,999',
                                                                        '  $20,000 to $24,999',
                                                                        '  $25,000 to $29,999',
                                                                        '  $30,000 to $34,999',
                                                                        '  $35,000 to $39,999',
                                                                        '  $40,000 to $44,999',
                                                                        '  $45,000 to $49,999',
                                                                        '  $50,000 to $59,999',
                                                                        '  $60,000 to $69,999',
                                                                        '  $70,000 to $79,999',
                                                                        '  $80,000 to $89,999',
                                                                        '  $90,000 to $99,999',
                                                                        '  $100,000 and over',
                                                                        )].tolist()

[]

In [244]:
# Slice demographics dataframe to obtain "Household Income" per Neighbourhood
income_data=neighbourhood_profile.iloc[lambda df: [0,1029, 1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069], 4:]
income_data.head()

,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,...,37,7,137,64,60,94,100,97,27,31
1029,Average after-tax income of households in 20...,"81,495","427,037","278,390","168,602","792,507","493,486","251,583","352,218","354,894",...,"272,986","412,302","629,030","240,272","147,462","239,484","222,648","541,217","302,358","213,860"
1054,Total - Household after-tax income groups in 2...,"1,112,930","9,115","8,140","4,620","15,940","12,125","6,090","15,070","9,525",...,"7,550","8,510","18,430","5,455","3,450","5,880","5,680","7,010","10,165","5,345"
1055,"Under $5,000","34,795",150,320,70,905,300,130,"2,545",610,...,575,105,435,70,60,135,215,240,350,115
1056,"$5,000 to $9,999","23,615",105,130,35,485,150,85,740,255,...,215,80,470,125,60,125,105,125,220,70


Cleaning up data...

In [245]:
# Drop irrelevant columns
income_data.rename(columns={'Characteristic':'Neighbourhood'}, inplace=True)
# Set index and Transpose
income_data=income_data.set_index('Neighbourhood').T
income_data.reset_index(inplace = True)
# Re-order columns
income_data.columns = ['Neighbourhood', 
                        'Neighbourhood ID', 
                        'Household After-tax Income', 
                        'Total Household Income Group',
                        'Under $5,000', 
                        '$5,000 to $9,999', 
                        '$10,000 to $14,999',
                        '$20,000 to $24,999',
                        '$25,000 to $29,999',
                        '$30,000 to $34,999',
                        '$35,000 to $39,999',
                        '$40,000 to $44,999',
                        '$45,000 to $49,999',
                        '$50,000 to $59,999',
                        '$60,000 to $69,999',
                        '$70,000 to $79,999',
                        '$80,000 to $89,999',
                        '$90,000 to $99,999',
                        '$100,000 and over']
# Set Neighbourhood ID and Unemployment Rate to numeric type
income_data.head()

,Neighbourhood,Neighbourhood ID,Household After-tax Income,Total Household Income Group,"Under $5,000","$5,000 to $9,999","$10,000 to $14,999","$20,000 to $24,999","$25,000 to $29,999","$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over"
0,City of Toronto,NaN,"81,495","1,112,930","34,795","23,615","37,080","55,930","51,910","52,205","53,515","55,545","55,770","53,385","98,910","86,755","75,120","63,360","51,570"
1,Agincourt North,129,"427,037","9,115",150,105,165,450,340,565,450,510,455,475,930,755,695,640,500
2,Agincourt South-Malvern West,128,"278,390","8,140",320,130,205,275,350,415,395,450,470,445,860,755,630,500,405
3,Alderwood,20,"168,602","4,620",70,35,80,85,160,175,170,210,185,225,390,355,365,310,265
4,Annex,95,"792,507","15,940",905,485,660,785,680,625,630,675,645,665,"1,160","1,060",820,750,620


### **PART 4 - Age Group (2016)**

We can extract the Population of Age Group from the following dataset (`Neighbourhood Profile`)

In [246]:
# Obtain the row number for "Population depending on age group" to allow us extract it from the dataframe
neighbourhood_profile.index[neighbourhood_profile['Characteristic'] == ('Children (0-14 years)', 'Youth (15-24 years)','Working Age (25-54 years)', 'Pre-retirement (55-64 years)', 'Seniors (65+ years)', 'Older Seniors (85+ years)')].tolist()

[]

In [247]:
# Slice demographics dataframe to obtain "Population depending on age group" per Neighbourhood
pop_data=neighbourhood_profile.iloc[lambda df: [0,9,10,11,12,13,14], 4:]
pop_data.head()

,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,...,37,7,137,64,60,94,100,97,27,31
9,Children (0-14 years),"398,135","3,840","3,075","1,760","2,360","3,605","2,325","1,695","2,415",...,"1,785","3,555","9,625","2,325","1,165","1,860","1,800","1,210","4,045","1,960"
10,Youth (15-24 years),"340,270","3,705","3,360","1,235","3,750","2,730","1,940","6,860","2,505",...,"2,230","2,625","7,660","1,035",675,"1,320","1,225",920,"4,750","1,870"
11,Working Age (25-54 years),"1,229,555","11,305","9,965","5,220","15,040","10,810","6,655","13,065","10,310",...,"7,480","8,140","21,945","6,165","3,790","6,420","5,860","5,960","12,290","5,860"
12,Pre-retirement (55-64 years),"336,670","4,230","3,265","1,825","3,480","3,555","2,030","1,760","2,540",...,"2,070","2,905","6,245","1,625","1,150","1,595","1,325","1,540","2,965","1,810"


Cleaning up data...

In [248]:
# Drop irrelevant columns
pop_data.rename(columns={'Characteristic':'Neighbourhood'}, inplace=True)
# Set index and Transpose
pop_data=pop_data.set_index('Neighbourhood').T
pop_data.reset_index(inplace = True)
# Re-order columns
pop_data.columns = ['Neighbourhood', 'Neighbourhood ID', 'Children (0-14 years)', 'Youth (15-24 years)', 'Working Age (25-54 years)', 'Pre-retirement (55-64 years)', 'Seniors (65+ years)', 'Older Seniors (85+ years)']
# Set Neighbourhood ID and Unemployment Rate to numeric type
pop_data.head()

,Neighbourhood,Neighbourhood ID,Children (0-14 years),Youth (15-24 years),Working Age (25-54 years),Pre-retirement (55-64 years),Seniors (65+ years),Older Seniors (85+ years)
0,City of Toronto,NaN,"398,135","340,270","1,229,555","336,670","426,945","66,000"
1,Agincourt North,129,"3,840","3,705","11,305","4,230","6,045",925
2,Agincourt South-Malvern West,128,"3,075","3,360","9,965","3,265","4,105",555
3,Alderwood,20,"1,760","1,235","5,220","1,825","2,015",320
4,Annex,95,"2,360","3,750","15,040","3,480","5,910","1,040"


## **4. Results - Cluster Data** <a name="results"></a>

### Cluster 1 - Crime Rate Vs. Unemployment Rate

In [249]:
cluster_data1 = pd.merge(crime_data, slice_neighbourhood_profile, on = ['Neighbourhood'])

# drop duplicate column
column = ["Neighbourhood ID_y"]       
cluster_data1.rename(columns={'Neighbourhood ID_x':'Neighbourhood ID'},inplace=True)
cluster_data1.drop(column, axis=1, inplace=True)  

cluster_data1.head()

,Neighbourhood,Neighbourhood ID,Population,Total Crime Rate,Unemployment Rate
0,Yonge-St.Clair,97,12528,646.5,5.9
1,York University Heights,27,27593,2642.0,10.7
2,Lansing-Westgate,38,16164,1020.9,7.2
3,Yorkdale-Glen Park,31,14804,2877.7,8.0
4,Stonegate-Queensway,16,25051,822.3,6.7


### Cluster 2 - Crime Rate Vs. Unemployment Rate Vs. Household Income

In [250]:
cluster_data2 = pd.merge(cluster_data1, income_data, on = ['Neighbourhood'])

# drop duplicate column
column = ["Neighbourhood ID_y"]       
cluster_data2.rename(columns={'Neighbourhood ID_x':'Neighbourhood ID'},inplace=True)
cluster_data2.drop(column, axis=1, inplace=True)  

cluster_data2.head()

,Neighbourhood,Neighbourhood ID,Population,Total Crime Rate,Unemployment Rate,Household After-tax Income,Total Household Income Group,"Under $5,000","$5,000 to $9,999","$10,000 to $14,999",...,"$30,000 to $34,999","$35,000 to $39,999","$40,000 to $44,999","$45,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over"
0,Yonge-St.Clair,97,12528,646.5,5.9,"541,217","7,010",240,125,190,...,250,280,310,330,315,625,475,450,335,305
1,York University Heights,27,27593,2642.0,10.7,"302,358","10,165",350,220,350,...,580,650,645,635,565,"1,065",870,715,575,435
2,Lansing-Westgate,38,16164,1020.9,7.2,"356,122","6,585",350,135,160,...,260,240,285,255,275,485,420,415,345,315
3,Yorkdale-Glen Park,31,14804,2877.7,8.0,"213,860","5,345",115,70,120,...,275,300,335,305,285,500,445,365,290,270
4,Stonegate-Queensway,16,25051,822.3,6.7,"516,575","10,380",210,115,215,...,440,460,450,435,425,815,720,590,575,495


### Cluster 3 - Crime Rate Vs. Unemployment Rate Vs. Household Income Vs. Age Group

In [251]:
cluster_data3 = pd.merge(cluster_data2, pop_data, on = ['Neighbourhood'])

# drop duplicate column
column = ["Neighbourhood ID_y"]        
cluster_data3.rename(columns={'Neighbourhood ID_x':'Neighbourhood ID'},inplace=True)
cluster_data3.drop(column, axis=1, inplace=True)  

cluster_data3.head()

,Neighbourhood,Neighbourhood ID,Population,Total Crime Rate,Unemployment Rate,Household After-tax Income,Total Household Income Group,"Under $5,000","$5,000 to $9,999","$10,000 to $14,999",...,"$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over",Children (0-14 years),Youth (15-24 years),Working Age (25-54 years),Pre-retirement (55-64 years),Seniors (65+ years),Older Seniors (85+ years)
0,Yonge-St.Clair,97,12528,646.5,5.9,"541,217","7,010",240,125,190,...,475,450,335,305,"1,210",920,"5,960","1,540","2,905",470
1,York University Heights,27,27593,2642.0,10.7,"302,358","10,165",350,220,350,...,870,715,575,435,"4,045","4,750","12,290","2,965","3,530",400
2,Lansing-Westgate,38,16164,1020.9,7.2,"356,122","6,585",350,135,160,...,420,415,345,315,"2,420","2,105","7,590","1,910","2,160",360
3,Yorkdale-Glen Park,31,14804,2877.7,8.0,"213,860","5,345",115,70,120,...,445,365,290,270,"1,960","1,870","5,860","1,810","3,295",775
4,Stonegate-Queensway,16,25051,822.3,6.7,"516,575","10,380",210,115,215,...,720,590,575,495,"4,060","2,495","10,660","3,660","4,165",750


## **5. Results - Plotting Cluster Data** <a name="results"></a>

### Bubble Plot for Cluster Data #1

To help visualize the data, we need to merge the cluster data with corresponding borough

In [252]:
column = ["PostalCode", "Latitude", "Longitude"]        #drop unecessary column
df.drop(column, axis=1, inplace=True) 

# merge two tables based on neighbourhood
bubble_data = pd.merge(cluster_data1,df, on='Neighbourhood')
bubble_data.head()

,Neighbourhood,Neighbourhood ID,Population,Total Crime Rate,Unemployment Rate,Borough
0,The Beaches,63,21567,848.5,5.7,East Toronto
1,Thorncliffe Park,55,21108,819.6,12.6,East York
2,Humewood-Cedarvale,106,14365,710.0,7.1,York
3,Scarborough Village,139,16724,1572.6,13.0,Scarborough
4,Humber Summit,21,12416,2923.7,9.6,North York


In [253]:
bubble_chart = px.scatter(bubble_data,              # Dataframe
                        x="Unemployment Rate",      # Column name for x-values
                        y="Total Crime Rate",       # Column name for y-values
                        size="Population",          # column name for size of bubble
                        color="Borough",            # Column name for Legend
                        hover_data=({
                        'Unemployment Rate': ':.2f', # Set the no. of decimal places
                        'Total Crime Rate': ':.2f',
                        'Population': ':.2f'}),
                        hover_name="Neighbourhood",
                        size_max=45,                 # maximum bubble size
                        title='Toronto Neighbourhoods - Crime Rate Vs. Unemployment Rate')

bubble_chart.update_layout(width=1000, height=800,)
bubble_chart.show()